In [155]:
import snowflake.connector
import yaml
import src.policy_helper as policy_helper
import time

ModuleNotFoundError: No module named 'src.policy_helper'

In [156]:
query_config = yaml.safe_load(open('../config.yaml', 'r'))


In [9]:
release_config

[{'VSR': '2.9.0',
  'vdrive': '52.0.13',
  'reecu': '8.1.1',
  'vehicle_sec': '3.2.0',
  'telestation_sec': '1.15.0',
  'depb': '2.3.0'},
 {'VSR': '2.10.2',
  'vdrive': '53.0.5',
  'reecu': '8.1.2',
  'vehicle_sec': '3.2.0',
  'telestation_sec': '1.15.0',
  'depb': '2.3.0'},
 {'VSR': '2.11.0',
  'vdrive': '54.0.5',
  'reecu': '8.1.2',
  'vehicle_sec': '3.2.0',
  'telestation_sec': '1.15.0',
  'depb': '2.3.0'}]

In [157]:
release_config = yaml.safe_load(open('../expected_config.yaml', 'r'))

In [166]:
query_config

{'user': 'jan.lack@vay.io',
 'authenticator': 'externalbrowser',
 'account': 'vay-vay_ee',
 'warehouse': 'BI_XS',
 'database': 'DWH'}

In [195]:
con = snowflake.connector.connect(
    user=query_config['user'],
    authenticator=query_config['authenticator'],
    account=query_config['account'],
    warehouse=query_config['warehouse'],
    database=query_config['database'],
    session_parameters={
        'QUERY_TAG': 'Get ODD ID by name'
    }
)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01dz3puk&SAMLRequest=nZJdb9owFIb%2FSuRdJ3ZSQGAREAWhsbWUlnTadmecA1g4duaPBvrrZ0KRuov2YheRIuc5x8%2FJe4bjYyWjFzBWaJWjNCEoAsV1KdQuR8%2FFPO6jyDqmSia1ghydwKLxaGhZJWs68W6vnuCPB%2Bui0EhZ2n7IkTeKamaFpYpVYKnjdD25v6NZQmhttNNcS%2FSu5PMKZi0YFwyvJaUVQW%2FvXE0xbpomaW4SbXY4I4RgMsCBOiNfrvwxzPQBn2LSOfOBCPjqze1WqMsv%2BExrc4Es%2FVoUq3j1sC5QNLmqTrWyvgKzBvMiODw%2F3V0EbDB4KAaDfpYl4GMOyhkm4zSxSjdbyQ7AdVV7Fxon4Q1vocRS70SYfTHLUX0QZeH2dU3uORz6le%2F2l99%2BnnrfH%2F1ys2EzQh5%2Fze9fl7er5WR77HMU%2FbiGm53DXVjrYaHOkbpwRLJOTNI46xZpSrs9mvWSTkZ%2Bo2gWIhWKubby6s041145m%2By03klo%2FTQ%2B55dhUdbj8Igyn5K0fL2p%2FQFd9oS2d5rRf0w%2FxO8bvG3dMgSxmK20FPwUzbWpmPs4pzRJ2xNRxtsWpVAxISdlacDakJeUupkaYC4stzMeEB5dbv13vUd%2FAQ%3D%3D&RelayState=51076 to 

In [179]:
con

In [196]:
release_config_query = """
WITH ranked_versions AS (
  SELECT
    vay_system_release,
    vdrive_major_version,
    vdrive_minor_version,
    vdrive_patch_version,
    depb_right_version_string,
    vreecu_version,
    ROW_NUMBER() OVER (PARTITION BY vay_system_release
                       ORDER BY vdrive_patch_version ASC) AS row_num
  FROM dwh.core_driving.sessions
  WHERE vay_system_release IS NOT NULL AND depb_right_version_string IS NOT NULL
)
SELECT
  SPLIT_PART(vay_system_release, '-', 2) as vay_system_release,
  vdrive_major_version,
  vdrive_minor_version,
  vdrive_patch_version,
  depb_right_version_string,  
  vreecu_version
FROM ranked_versions
QUALIFY row_num = 1
ORDER BY 
  CAST(SPLIT_PART(vay_system_release, '.', 1) AS INTEGER) DESC,
  CAST(SPLIT_PART(vay_system_release, '.', 2) AS INTEGER) DESC,
  CAST(SPLIT_PART(vay_system_release, '.', 3) AS INTEGER) DESC
LIMIT 3;

"""

In [197]:
release_configs = con.cursor().execute(release_config_query).fetchall()

In [184]:
release_configs

[('2.12.0', 55, 0, 1, '2.3.0', '8.2.0'),
 ('2.11.0', 54, 0, 3, '2.3.0', '8.1.2'),
 ('2.10.0', 53, 0, 3, '2.3.0', '8.1.2')]

In [198]:

release_configurations = []
for row in release_configs:
    release = {}
    release['vay_system_release'] = row[0]
    release['vdrive_major_version'] = row[1]
    release['vdrive_minor_version'] = row[2]
    release['vdrive_patch_version'] = row[3]
    release['depb'] = row[4]
    release['vreecu_version'] = row[5]
    release_configurations.append(release)

release_configurations

[{'vay_system_release': '2.12.0',
  'vdrive_major_version': 55,
  'vdrive_minor_version': 0,
  'vdrive_patch_version': 1,
  'depb': '2.3.0',
  'vreecu_version': '8.2.0'},
 {'vay_system_release': '2.11.0',
  'vdrive_major_version': 54,
  'vdrive_minor_version': 0,
  'vdrive_patch_version': 3,
  'depb': '2.3.0',
  'vreecu_version': '8.1.2'},
 {'vay_system_release': '2.10.0',
  'vdrive_major_version': 53,
  'vdrive_minor_version': 0,
  'vdrive_patch_version': 3,
  'depb': '2.3.0',
  'vreecu_version': '8.1.2'}]

In [199]:
vehicle_query = """
select
  TO_CHAR(_ingested_at, 'YYYY-MM-DD HH24:MI') AS "Session started UTC"
  , vehicle:name as "name"
  , CAST(vehicle:software_version:sem_ver_tag:major AS INTEGER) as "vdrive_major_version"
  , CAST(vehicle:software_version:sem_ver_tag:minor AS INTEGER) as "vdrive_minor_version"
  , CAST(vehicle:software_version:sem_ver_tag:patch AS INTEGER) as "vdrive_patch_version"
  , concat(vehicle:depb_left_version:major, '.', vehicle:depb_left_version:minor, '.', vehicle:depb_left_version:patch) as "depb_left"
  , concat(vehicle:depb_right_version:major, '.', vehicle:depb_right_version:minor, '.', vehicle:depb_right_version:patch) as "depb_right"
  , concat(vehicle:reecu:mcu_fw_version:major, '.', vehicle:reecu:mcu_fw_version:minor, '.', vehicle:reecu:mcu_fw_version:patch) as "vreecu_version"
from dwh.raw_driving.session_init
where true
  and vehicle:name LIKE any ('ve-0%', 've-us-%')
  and _ingested_at >= CURRENT_DATE() - INTERVAL '30 DAY'
  and concat(vehicle:depb_right_version:major, '.', vehicle:depb_right_version:minor, '.', vehicle:depb_right_version:patch) is NOT NULL
  and concat(vehicle:reecu:mcu_fw_version:major, '.', vehicle:reecu:mcu_fw_version:minor, '.', vehicle:reecu:mcu_fw_version:patch) is NOT NULL
  and concat(vehicle:software_version:sem_ver_tag:major, '.', vehicle:software_version:sem_ver_tag:minor, '.', vehicle:software_version:sem_ver_tag:patch) is NOT NULL
  QUALIFY 
  ROW_NUMBER() OVER (PARTITION BY vehicle:name ORDER BY _ingested_at DESC) = 1
  ORDER BY vehicle:name ASC
LIMIT 50
"""

telestaion_query = """
select
  TO_CHAR(_ingested_at, 'YYYY-MM-DD HH24:MI') AS "Session started UTC"
  , telestation:name as "name"
  , cast(telestation:software_version:sem_ver_tag:major as integer) as "vdrive_major_version"
  , cast(telestation:software_version:sem_ver_tag:minor as integer) as "vdrive_minor_version"
  , cast(telestation:software_version:sem_ver_tag:patch as integer) as "vdrive_patch_version"
  , concat(telestation:reecu:mcu_fw_version:major, '.', telestation:reecu:mcu_fw_version:minor, '.', telestation:reecu:mcu_fw_version:patch) as "vreecu_version"
from dwh.raw_driving.session_init
where true
  and telestation:name LIKE any ('ts-ber-0%', 'ts-las-0%', 'ts-gd%', 'ts-ham-0%')
  and _ingested_at >= CURRENT_DATE() - INTERVAL '30 DAY'
  and concat(telestation:reecu:mcu_fw_version:major, '.', telestation:reecu:mcu_fw_version:minor, '.', telestation:reecu:mcu_fw_version:patch) is NOT NULL
  and concat(telestation:software_version:sem_ver_tag:major, '.', telestation:software_version:sem_ver_tag:minor, '.', telestation:software_version:sem_ver_tag:patch) is NOT NULL
  QUALIFY 
  ROW_NUMBER() OVER (PARTITION BY telestation:name ORDER BY _ingested_at DESC) = 1
  ORDER BY vehicle:name ASC
LIMIT 50
"""


In [200]:
vehicle_output = con.cursor().execute(vehicle_query).fetchall()

In [201]:
vehicle_output

[('2024-01-23 13:35', '"ve-0008"', 55, 0, 1, '2.3.0', '2.3.0', '8.1.27559'),
 ('2024-01-23 12:32', '"ve-0009"', 56, 0, 0, '0.0.0', '2.3.0', '8.3.0'),
 ('2024-01-22 16:41', '"ve-0010"', 55, 0, 1, '0.0.0', '2.3.0', '8.2.0'),
 ('2024-01-24 13:45', '"ve-0011"', 56, 0, 0, '0.0.0', '2.3.0', '8.3.0'),
 ('2024-01-24 16:37', '"ve-0012"', 55, 0, 1, '0.0.0', '2.3.0', '8.2.0'),
 ('2024-01-09 15:18', '"ve-0014"', 54, 0, 7, '2.1.1', '2.1.1', '0.0.0'),
 ('2024-01-18 15:25', '"ve-0015"', 54, 0, 7, '0.0.0', '2.3.0', '8.1.2'),
 ('2024-01-24 11:02', '"ve-0020"', 55, 0, 1, '0.0.0', '2.4.0', '8.2.0'),
 ('2024-01-24 21:54', '"ve-us-0001"', 55, 0, 1, '0.0.0', '2.3.0', '8.2.0'),
 ('2024-01-24 23:10', '"ve-us-0002"', 55, 0, 1, '0.0.0', '2.3.0', '8.2.0'),
 ('2024-01-24 22:34', '"ve-us-0003"', 54, 0, 7, '2.3.0', '2.3.0', '8.1.2'),
 ('2024-01-18 21:51', '"ve-us-0005"', 54, 0, 7, '2.3.0', '2.3.0', '8.1.2'),
 ('2024-01-24 22:34', '"ve-us-0007"', 54, 0, 7, '2.3.0', '2.3.0', '8.1.2'),
 ('2024-01-16 18:38', '"ve-us-00

In [202]:
telestation_output = con.cursor().execute(telestaion_query).fetchall()

In [203]:
vehicle_configuration = {}

for vehicle in vehicle_output:
    vehicle_configuration[vehicle[1].replace('"', '')] = {
        'vdrive_major_version': vehicle[2],
        'vdrive_minor_version': vehicle[3],
        'vdrive_patch_version': vehicle[4],
        'depb_left': vehicle[5],
        'depb_right': vehicle[6],
        'vreecu_version': vehicle[7],
        'date': vehicle[0]
    }

telestation_configuration = {}
for telestation in telestation_output:
    telestation_configuration[telestation[1].replace('"', '')] = {
        'vdrive_major_version': vehicle[2],
        'vdrive_minor_version': vehicle[3],
        'vdrive_patch_version': vehicle[4],
        'vreecu_version': telestation[5],
    }


In [206]:
vehicle_configuration['ve-0010']

{'vdrive_major_version': 55,
 'vdrive_minor_version': 0,
 'vdrive_patch_version': 1,
 'depb_left': '0.0.0',
 'depb_right': '2.3.0',
 'vreecu_version': '8.2.0',
 'date': '2024-01-22 16:41'}

In [205]:
release_configurations[0]

{'vay_system_release': '2.12.0',
 'vdrive_major_version': 55,
 'vdrive_minor_version': 0,
 'vdrive_patch_version': 1,
 'depb': '2.3.0',
 'vreecu_version': '8.2.0'}

In [217]:
mismatched_vehicles = {}
matched_vehicles = {}
vsr_keys = ["vdrive_major_version", "vdrive_minor_version", "vdrive_patch_version", "vreecu_version", "depb"]
vehicle_keys = ["vdrive_major_version", "vdrive_minor_version", "vdrive_patch_version", "vreecu_version", "depb_right"]
for vsr_config in release_configurations:
    mismatched_vehicles[vsr_config["vay_system_release"]] = []
    matched_vehicles[vsr_config["vay_system_release"]] = []
for vehicle_name, vehicle_params in vehicle_configuration.items():
    match_found = False
    for vsr_config in release_configurations:
        if (
            vehicle_params['vdrive_major_version'] == vsr_config['vdrive_major_version'] and
            vehicle_params['vdrive_minor_version'] == vsr_config['vdrive_minor_version'] and
            vehicle_params['vdrive_patch_version'] >= vsr_config['vdrive_patch_version'] and
            vehicle_params['vreecu_version'] == vsr_config['vreecu_version'] and
            vehicle_params['depb_right'] == vsr_config['depb']
        ):
            match_found = True
            matched_vehicles[vsr_config["vay_system_release"]].append(vehicle_name)
            break
        
    if not match_found:
        for vsr_config in release_configurations:
            mismatch_params = {
                vehicle_key: vehicle_params[vehicle_key] 
                for vsr_key, vehicle_key in zip(vsr_keys, vehicle_keys) 
                if vehicle_key != "vdrive_patch_version" and vehicle_params[vehicle_key] != vsr_config[vsr_key]
            }
            if vehicle_params['vdrive_patch_version'] < vsr_config['vdrive_patch_version']:
                mismatch_params['vdrive_patch_version'] = vehicle_params['vdrive_patch_version']
            mismatched_vehicles[vsr_config["vay_system_release"]].append({'name': vehicle_name, 'params': mismatch_params})

print("Mismatched Vehicles:")
print(mismatched_vehicles)
print("Matched Vehicles:")
print(matched_vehicles)

Mismatched Vehicles:
{'2.12.0': [{'name': 've-0008', 'params': {'vreecu_version': '8.1.27559'}}, {'name': 've-0009', 'params': {'vdrive_major_version': 56, 'vreecu_version': '8.3.0', 'vdrive_patch_version': 0}}, {'name': 've-0011', 'params': {'vdrive_major_version': 56, 'vreecu_version': '8.3.0', 'vdrive_patch_version': 0}}, {'name': 've-0014', 'params': {'vdrive_major_version': 54, 'vreecu_version': '0.0.0', 'depb_right': '2.1.1'}}, {'name': 've-0020', 'params': {'depb_right': '2.4.0'}}], '2.11.0': [{'name': 've-0008', 'params': {'vdrive_major_version': 55, 'vreecu_version': '8.1.27559', 'vdrive_patch_version': 1}}, {'name': 've-0009', 'params': {'vdrive_major_version': 56, 'vreecu_version': '8.3.0', 'vdrive_patch_version': 0}}, {'name': 've-0011', 'params': {'vdrive_major_version': 56, 'vreecu_version': '8.3.0', 'vdrive_patch_version': 0}}, {'name': 've-0014', 'params': {'vreecu_version': '0.0.0', 'depb_right': '2.1.1'}}, {'name': 've-0020', 'params': {'vdrive_major_version': 55, 'vr

In [216]:
mismatched_vehicles_string = ""
for release in mismatched_vehicles:
    mismatched_vehicles_string = mismatched_vehicles_string + "*" + release + "*" + "\n"
    for vehicle in mismatched_vehicles[release]:
        mismatched_vehicles_string = mismatched_vehicles_string + vehicle["name"] + "\n"
        for key in vehicle["params"]:
            mismatched_vehicles_string = mismatched_vehicles_string + " " + key + " " + str(vehicle["params"][key]) + "\n"
        mismatched_vehicles_string = mismatched_vehicles_string + "\n"

print(mismatched_vehicles_string)

*2.12.0*
ve-0008
 vreecu_version 8.1.27559

ve-0009
 vdrive_major_version 56
 vreecu_version 8.3.0
 vdrive_patch_version 0

ve-0011
 vdrive_major_version 56
 vreecu_version 8.3.0
 vdrive_patch_version 0

ve-0014
 vdrive_major_version 54
 vreecu_version 0.0.0
 depb_right 2.1.1

ve-0020
 depb_right 2.4.0

*2.11.0*
ve-0008
 vdrive_major_version 55
 vreecu_version 8.1.27559
 vdrive_patch_version 1

ve-0009
 vdrive_major_version 56
 vreecu_version 8.3.0
 vdrive_patch_version 0

ve-0011
 vdrive_major_version 56
 vreecu_version 8.3.0
 vdrive_patch_version 0

ve-0014
 vreecu_version 0.0.0
 depb_right 2.1.1

ve-0020
 vdrive_major_version 55
 vreecu_version 8.2.0
 depb_right 2.4.0
 vdrive_patch_version 1

*2.10.0*
ve-0008
 vdrive_major_version 55
 vreecu_version 8.1.27559
 vdrive_patch_version 1

ve-0009
 vdrive_major_version 56
 vreecu_version 8.3.0
 vdrive_patch_version 0

ve-0011
 vdrive_major_version 56
 vreecu_version 8.3.0
 vdrive_patch_version 0

ve-0014
 vdrive_major_version 54
 vreecu_v

In [218]:
mismatched_telestations = {}
matched_telestations = {}
vsr_keys = ["vdrive", "reecu"]
telestation_keys = ["telestation_vdrive", "telestation_reecu"]

for vsr_config in release_config:
    mismatched_telestations[vsr_config["VSR"]] = []
    matched_telestations[vsr_config["VSR"]] = []
    
for telestation_name, telestation_params in telestation_configuration.items():
    match_found = False
    for vsr_config in release_config:
        if (
            telestation_params['vdrive_major_version'] == vsr_config['vdrive_major_version'] and
            telestation_params['vdrive_minor_version'] == vsr_config['vdrive_minor_version'] and
            telestation_params['vdrive_patch_version'] >= vsr_config['vdrive_patch_version'] and
            telestation_params['vreecu_version'] == vsr_config['vreecu_version'] and
        ):
            match_found = True
            matched_telestations[vsr_config["VSR"]].append(telestation_name)
            break
    if match_found == False:
        for vsr_config in release_config:
            mismatched_telestations[vsr_config["VSR"]].append({
                'name': telestation_name,
                'params': {telestation_key: telestation_params[telestation_key] for vsr_key, telestation_key in zip(vsr_keys, telestation_keys) if telestation_params[telestation_key] != vsr_config[vsr_key]}
            })

print("Mismatched Telestations:")
print(mismatched_telestations)
print("Matched Telestations:")
print(matched_telestations)

KeyError: 'telestation_vdrive'

In [213]:
mismatched_telestations = {}
matched_telestations = {}
vsr_keys = ["vdrive_major_version", "vdrive_minor_version", "vdrive_patch_version", "vreecu_version"]
telestation_keys = ["vdrive_major_version", "vdrive_minor_version", "vdrive_patch_version", "vreecu_version"]

for vsr_config in release_configurations:
    mismatched_telestations[vsr_config["vay_system_release"]] = []
    matched_telestations[vsr_config["vay_system_release"]] = []

for telestation_name, telestation_params in telestation_configuration.items():
    match_found = False
    for vsr_config in release_configurations:
        if (
            telestation_params['vdrive_major_version'] == vsr_config['vdrive_major_version'] and
            telestation_params['vdrive_minor_version'] == vsr_config['vdrive_minor_version'] and
            telestation_params['vdrive_patch_version'] >= vsr_config['vdrive_patch_version'] and
            telestation_params['vreecu_version'] == vsr_config['vreecu_version']
        ):
            match_found = True
            matched_telestations[vsr_config["vay_system_release"]].append(telestation_name)
            break

    if not match_found:
        for vsr_config in release_configurations:
            mismatch_params = {
                telestation_keys: telestation_params[telestation_keys] 
                for vsr_key, telestation_key in zip(vsr_keys, telestation_keys) 
                if telestation_key != "vdrive_patch_version" and vehicle_params[telestation_key] != vsr_config[vsr_key]
            }
            if telestation_params['vdrive_patch_version'] < vsr_config['vdrive_patch_version']:
                mismatch_params['vdrive_patch_version'] = telestation_params['vdrive_patch_version']
            mismatched_telestations[vsr_config["vay_system_release"]].append({'name': telestation_name, 'params': mismatch_params})

print("Mismatched Telestations:")
print(mismatched_telestations)
print("Matched Telestations:")
print(matched_telestations)

TypeError: unhashable type: 'list'

In [ ]:
def misconfiguration_string(mismatched_configuraiton: dict, ):
    mismatched_vehicles_string = ""
    for release in mismatched_configuraiton:
        mismatched_vehicles_string = mismatched_vehicles_string + "*" + release + "*" + "\n"
        for system in mismatched_configuraiton[release]:
            mismatched_vehicles_string = mismatched_vehicles_string + system["name"] + "\n"
            for key in system["params"]:
                mismatched_vehicles_string = mismatched_vehicles_string + " " + key + " " + system["params"][key] + "\n"
            mismatched_vehicles_string = mismatched_vehicles_string + "\n"

    return mismatched_vehicles_string